<a href="https://colab.research.google.com/github/fazekas-gergo/composer-ai/blob/develop%2Ffirst-steps/ComposerAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Composer AI
---

